In [14]:
import pandas as pd
from multiprocess import Pool
import itertools
from tqdm import tqdm

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

In [9]:
processed = pd.read_parquet('processed.parquet').to_dict(orient = 'records')
processed = [(i, processed[i]) for i in range(len(processed))]

In [18]:
def loop(rows):
    rows, _ = rows
    accept = []
    for row in tqdm(rows):
        k, row = row
        alignment = row['alignment']
        failed = False
        for i in range(len(alignment) - 1):
            if alignment[i]['text'][-1] in ',.?!':
                if (alignment[i + 1]['start'] - alignment[i]['end']) < 0.1:
                    failed = True
                    break
        if failed:
            continue

        accept.append(k)
    return accept

In [22]:
accept = loop((processed, 0))

100%|██████████| 1645455/1645455 [00:11<00:00, 143674.78it/s]


In [23]:
len(accept)

977091

In [42]:
import json

with open('accept-period.json', 'w') as fopen:
    json.dump(accept, fopen)